In [1]:
import torch



In [2]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [3]:
w_q=torch.nn.Linear(inputs.shape[1],2,bias=False)
print(w_q)
w_k=torch.nn.Linear(inputs.shape[1],2,bias=False)
w_v=torch.nn.Linear(inputs.shape[1],2,bias=False)

Linear(in_features=3, out_features=2, bias=False)


In [4]:
queries=w_q(inputs)
keys=w_k(inputs)
values=w_v(inputs)

print(queries)

tensor([[-0.0557,  0.3296],
        [ 0.3878,  0.5322],
        [ 0.3798,  0.5047],
        [ 0.2754,  0.3604],
        [ 0.1288, -0.1322],
        [ 0.3635,  0.6427]], grad_fn=<MmBackward0>)


In [5]:
attn_score=queries@keys.T
print(attn_score)

tensor([[-0.1139, -0.1506, -0.1426, -0.0993,  0.0417, -0.1816],
        [-0.0469, -0.2759, -0.2640, -0.1998,  0.0241, -0.3223],
        [-0.0411, -0.2625, -0.2512, -0.1905,  0.0218, -0.3064],
        [-0.0281, -0.1877, -0.1797, -0.1363,  0.0151, -0.2190],
        [ 0.0762,  0.0531,  0.0497,  0.0311, -0.0264,  0.0664],
        [-0.0868, -0.3260, -0.3114, -0.2326,  0.0386, -0.3828]],
       grad_fn=<MmBackward0>)


In [6]:
attn_weight=torch.softmax(attn_score/keys.shape[1]**0.5,dim=1)
print(attn_weight)

tensor([[0.1657, 0.1615, 0.1624, 0.1674, 0.1850, 0.1580],
        [0.1825, 0.1552, 0.1565, 0.1638, 0.1919, 0.1502],
        [0.1821, 0.1557, 0.1570, 0.1639, 0.1904, 0.1510],
        [0.1778, 0.1589, 0.1598, 0.1647, 0.1834, 0.1554],
        [0.1707, 0.1680, 0.1676, 0.1654, 0.1588, 0.1696],
        [0.1817, 0.1534, 0.1550, 0.1639, 0.1985, 0.1474]],
       grad_fn=<SoftmaxBackward0>)


# Create Mask where the value above the diagonal is zero


In [7]:
#use pytorch tril function to diagonalize
context_length=inputs.shape[0]
mask1=torch.tril(torch.ones(context_length,context_length))
print(mask1)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [8]:
#use element wise multiplication not matrix
masked_attn_score=attn_weight*mask1
print(masked_attn_score)

#but this attention score is not fully masked or optimal becaus the softmax normalization adds influence on its from the previous attn score so use upper negative infinity mask instead of zero

tensor([[0.1657, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1825, 0.1552, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1821, 0.1557, 0.1570, 0.0000, 0.0000, 0.0000],
        [0.1778, 0.1589, 0.1598, 0.1647, 0.0000, 0.0000],
        [0.1707, 0.1680, 0.1676, 0.1654, 0.1588, 0.0000],
        [0.1817, 0.1534, 0.1550, 0.1639, 0.1985, 0.1474]],
       grad_fn=<MulBackward0>)


In [9]:
#normalize the masked attention score  to sum up 1 in each row to get masked attn weight
row_sum=masked_attn_score.sum(dim=1,keepdim=True)
print(row_sum)
masked_attn_weight=masked_attn_score/row_sum
print(masked_attn_weight)

tensor([[0.1657],
        [0.3377],
        [0.4948],
        [0.6612],
        [0.8304],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5404, 0.4596, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3681, 0.3147, 0.3172, 0.0000, 0.0000, 0.0000],
        [0.2690, 0.2403, 0.2416, 0.2491, 0.0000, 0.0000],
        [0.2056, 0.2023, 0.2018, 0.1991, 0.1912, 0.0000],
        [0.1817, 0.1534, 0.1550, 0.1639, 0.1985, 0.1474]],
       grad_fn=<DivBackward0>)


In [10]:
masked_context_vectore=masked_attn_weight@values
print(masked_context_vectore)

tensor([[ 0.1667, -0.2088],
        [ 0.2543, -0.4122],
        [ 0.2803, -0.4893],
        [ 0.2765, -0.4624],
        [ 0.1975, -0.4792],
        [ 0.2208, -0.4564]], grad_fn=<MmBackward0>)


# Negative infinity masking to avoid any influence of future attention

In [11]:
print(attn_score)

tensor([[-0.1139, -0.1506, -0.1426, -0.0993,  0.0417, -0.1816],
        [-0.0469, -0.2759, -0.2640, -0.1998,  0.0241, -0.3223],
        [-0.0411, -0.2625, -0.2512, -0.1905,  0.0218, -0.3064],
        [-0.0281, -0.1877, -0.1797, -0.1363,  0.0151, -0.2190],
        [ 0.0762,  0.0531,  0.0497,  0.0311, -0.0264,  0.0664],
        [-0.0868, -0.3260, -0.3114, -0.2326,  0.0386, -0.3828]],
       grad_fn=<MmBackward0>)


In [12]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked=attn_score.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[-0.1139,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.0469, -0.2759,    -inf,    -inf,    -inf,    -inf],
        [-0.0411, -0.2625, -0.2512,    -inf,    -inf,    -inf],
        [-0.0281, -0.1877, -0.1797, -0.1363,    -inf,    -inf],
        [ 0.0762,  0.0531,  0.0497,  0.0311, -0.0264,    -inf],
        [-0.0868, -0.3260, -0.3114, -0.2326,  0.0386, -0.3828]],
       grad_fn=<MaskedFillBackward0>)


In [22]:
print(attn_score*mask1)

tensor([[-0.1139, -0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0469, -0.2759, -0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0411, -0.2625, -0.2512, -0.0000,  0.0000, -0.0000],
        [-0.0281, -0.1877, -0.1797, -0.1363,  0.0000, -0.0000],
        [ 0.0762,  0.0531,  0.0497,  0.0311, -0.0264,  0.0000],
        [-0.0868, -0.3260, -0.3114, -0.2326,  0.0386, -0.3828]],
       grad_fn=<MulBackward0>)


In [14]:
masked_attn_weight=torch.softmax(masked/keys.shape[1],dim=1)
print(masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5286, 0.4714, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3577, 0.3202, 0.3220, 0.0000, 0.0000, 0.0000],
        [0.2633, 0.2431, 0.2441, 0.2495, 0.0000, 0.0000],
        [0.2040, 0.2016, 0.2013, 0.1994, 0.1938, 0.0000],
        [0.1774, 0.1574, 0.1585, 0.1649, 0.1888, 0.1530]],
       grad_fn=<SoftmaxBackward0>)


# Masking additional attention weight with Dropout

In [15]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
print(dropout)
ones=torch.ones(6,6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [16]:
attn_dropout=dropout(attn_weight)
print(attn_dropout)

tensor([[0.3314, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3130, 0.0000, 0.3837, 0.3004],
        [0.0000, 0.3114, 0.3139, 0.3277, 0.3808, 0.3019],
        [0.0000, 0.0000, 0.3195, 0.3295, 0.0000, 0.3108],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3176, 0.3391],
        [0.3634, 0.3069, 0.0000, 0.3278, 0.3971, 0.2948]],
       grad_fn=<MulBackward0>)


# Causal attention Class with Batches

In [17]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [18]:
print(batch.transpose(1,2))#change row(1) with colmn(2)

tensor([[[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]],

        [[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]]])


In [19]:
class Causal_Attention(torch.nn.Module):
    def __init__(self,in_d,out_d,context_length,dropout_percent):
        super().__init__()
        self.w_q=torch.nn.Linear(in_d,out_d)
        self.w_k=torch.nn.Linear(in_d,out_d)
        self.w_v=torch.nn.Linear(in_d,out_d)
        self.dropout=torch.nn.Dropout(dropout_percent)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    
    def forward(self,input_embedding):
        
        b,num_token,d_in=input_embedding.shape
        queries=self.w_q(input_embedding)
        keys=self.w_k(input_embedding)
        values=self.w_v(input_embedding)

        print(keys.shape[-1])

        attn_score=queries@keys.transpose(1,2)

        attn_score.masked_fill_(self.mask.bool()[:num_token, :num_token],-torch.inf)
        
        print('attn_score',attn_score)
        attn_weight=torch.softmax(attn_score/keys[-1]**0.5,dim=-1)
        print('attn_weight',attn_weight)
        attn_weight=self.dropout(attn_weight)

        masked_context_vectore=attn_weight@values
        return masked_context_vectore

        
        

        
        


In [20]:
in_d,out_d=3,2
print(in_d)

3


In [21]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
ca=Causal_Attention(in_d,out_d,context_length,0.0)
print(ca)
context_vec=ca.forward(batch)

6
Causal_Attention(
  (w_q): Linear(in_features=3, out_features=2, bias=True)
  (w_k): Linear(in_features=3, out_features=2, bias=True)
  (w_v): Linear(in_features=3, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
2
attn_score tensor([[[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]],

        [[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]]],
       grad_fn=<MaskedFillBackward0>

RuntimeError: The size of tensor a (6) must match the size of tensor b (2) at non-singleton dimension 2

In [ ]:
x=[[1,2,3],[4,5,6],[7,8,9]]
y=torch.zeros(3,3)
for r in range(3):
    row=x[r]
    row[r+1:y.append(row)]=row[r+1:]*0
    y[r][r]
    
print(y)

   



tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
